In [ ]:
"""
computes quantities associated with the Gaussian linear state space model.
"""

from textwrap import dedent
import numpy as np
from numpy.random import multivariate_normal
from scipy.linalg import solve
from numba import jit

@jit
def simulate_linear_model(A, x0, v, ts_length):
    """
    this is a separate function for simulating a vector linear system of
    the form
    
        x_{t+1} = A x_t + v_t given x_0 = x0
        
    here, x_t and v_t are both n x 1 and A is n x n.
    
    the purpose of separating this functionality out is to target it for
    optimization by Numba. For the same reason, matrix multiplication is
    broken down into for loops.
    
    parameters
    ------
    A: array_like or scalar(float)
        should be n x n
    x0: array_like
        should be n x 1. initial condition
    v: np.ndarray
        should be n x ts_length-1. its t-th column is used as the time t
        shock v_t
    ts_length: int
        the length of the time series
        
    returns
    ------
    x: np.ndarray
        time series with ts_length columns, the t-th column being x_t.
    """
    A = np.asarray(A)
    n = A.shape[0]
    x = np.empty((n, ts_length))
    x[:, 0] = x0
    for t in range(ts_length-1):
        # x[:, t+1] = A.dot(x[:, t]) + v[:, t]
        for i in range(n):
            x[i, t+1] = v[i, t] #shock
            for j in range(n):
                x[i, t+1] += A[i, j] * x[j, t] #dot product
    return x

class LinearStateSpace(object):
    """
    a class that describes a Gaussian linear state space model of the
    form;
    
        x_{t+1} = A x_t + C w_{t+1}
        
        y_t = G x_t + H v_t
        
    where {w_t}  and {v_t} are independent and standard normal with dimensions
    k and l respectively. The initial conditions are mu_0 and Sigma_0 for x_0
    ~ N(mu_0, Sigma_0). when Sigma_0 = 0, the draw of x_0 is exactly mu_0.
    
    parameters
    ------
    A: array_like or scalar(float)
        part of the state transition equation. It should be n x n
    C: array_like or scalar(float)
        part of the state transition equation. It should be n x m
    G: array_like or scalar(float)
        part of the observation equation. It should be k x n
    H: array_like or scalar(float), optional(default=None)
        part of the observation equation. It should be k x l
    mu_0: array_like or scalar(float), optional(default=None)
        this is the mean of initial draw and is n x 1
    Sigma_0: array_like or scalar(float), optional(default=None)
        this is the variance of the initial draw and is n x n and
        also should be positive definite and symmetric
        
    attributes
    ------
    A, C, G, H, mu_0, Sigma_0: see parameters
    n, k, m, l: scalar(int)
        the dimensions of x_t, y_t, w_t, and v_t respetively
        
    """
    
    def __int__(self, A, C, G, H=None, mu_0=None, Sigma_0=None):
        self.A, self.G, self.C = list(map(self.convert, (A, G, C)))
        #-check input shapes-#
        ni, nj = self.A.shape
        if ni != nj:
            raise ValueError("matrix A (shape: %s) needs to be square" % (self.A.shape))
        if ni != self.C.shape[0]:
            raise ValueError("matrix C (shape: %s) does not have compatible dimensions with A. It should be shape: %s" % (self.C.shape, (ni, 1)))
        self.m =self.C.shape[1]
        self.k, self.n =self.G.shape
        if self.n !=ni:
            raise ValueError("matrix G (shape: %s) does not have compatible dimensions with A (%s)" %(self.G.shape, self.A.shape))
        if H is None:
            self.H = None
            self.l = None
        else:
            self.H = self.convert(H)
            self.l = self.H.shape[1]
        if mu_0 is None:
            self.mu_0 = np.zeros((self.n, 1))
        else:
            self.mu_0 = self.convert(mu_0)
            self.mu_0.shape = self.n, 1
        if Sigma_0 is None:
            self.Sigma_0 = np.zeros((self.n, self.n))
        else:
            self.Sigma_0 = self.convert(Sigma_0)
            
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        m="""\
        Linear Gaussian state space model:
            -dimension of state space: {n}
            -number of innovations: {m}
            -dimension of observation equation: {k}
        """
        return dedent(m.format(n=self.n, k=self.k, m=self.m))
    
    def convert(self, x):
        """
        convert array_like objects (lists of lists, floats, etc.) into
        well formed 2D NumPy arrays
        
        """
        return np.atleast_2d(np.asarray(x, dtype = 'float'))
    
    def simulate(self, ts_length=100):
        """
        simulate a time series of length ts_length, first drawing
        
            x_0 ~ N(mu_0, Sigma_0)
            
        parameters
        ------
        
        ts_length: scalar(int), optional(default=100)
            the length of the simulation
            
        returns
        ------
        x: array_like(float)
            an n x ts_length array, where the t-th column is x_t
        y: array_like(float)
            a k x ts_length array, where the t-th column is y_t
            
        """
        x0 = multivariate_normal(self.mu_0.flatten(), self.Sigma_0)
        w = np.random.randn(self.m, ts_length-1)
        v = self.C.dot(w) #multiply each w_t by C to get v_t = C w_t
        #== simulate time series==#
        x = simulate_linear_model(self.A, x0, v, ts_length)
        
        if self.H is not None:
            v = np.random.randn(self.l, ts_length)
            y = self.G.dot(x) +self.H.dot(x)
        else:
            y = self.G.dot(x)
            
        return x, y
    
    def replicate(self, T=10, num_reps=100):
        """
        simulate num_reps observations of x_T and y_T given
        x_0 ~ N(mu_0, Sigma_0).
        
        parameters
        ------
        T: scalar(int), optional(default=10)
            the period that we want to replicate values for
        num_reps: scalar(int), optional(default=100)
            the number of replications that we want
            
        returns
        ------
        x: array_like(float)
            an n x num_reps array, where the j-th column is the j_th
            observation of x_T
        
        y: array_like(float)
            a k x num_reps array, where the j-th column is the j_th
            observation of y_T
            
        """
        x = np.empty((self.n, num_reps))
        for j in range(num_reps):
            x_T, _ = self.simulate(ts_length=T+1)
            x[:, j] = x_T[:, -1]
        if self.H is not None:
            v = np.random.randn(self.l, num_reps)
            y = self.G.dot(x) + self.H.dot(v)
        else:
            y = self.G.dot(x)
            
        return x, y
    
    def moment_sequence(self):
        """
        create a generator to calculate the population mean and
        variance-covariance matrix for both x_t and y_t, starting at
        the initial condition (self.mu_0, self.Sigma_0). each iteration
        produces a 4-tuple of items (mu_x, mu_y, Sigma_x, Sigma_y) for
        the next period.
        
        yields
        ------
        mu_x: array_like(float)
            an n x 1 array representing the population mean of x_t
        mu_y: array_like(float)
            a k x 1 array representing the population mean of y_t
        Sigma_x: array_like(float)
            an n x n array representing the variance-covariance matrix
            of x_t
        Sigma_y: array_like(float)
            an k x k array representing the variance-covariance matrix
            of y_t
            
        """
        #==simplify names==#
        A, C, G, H =self.A, self.C, self.G, self.H
        #==initial moments==#
        mu_x, Sigma_x = self.mu_0, self.Sigma_0
        
        while 1:
            mu_y = G.dot(mu_x)
            if H is None:
                Sigma_y = G.dot(Sigma_x).dot(G.T)
            else:
                Sigma_y = G.dot(Sigma_x).dot(G.T) + H.dot(H.T)
            
            yield mu_x, mu_y, Sigma_x, Sigma_y
            
            #==update moments of x==#
            mu_x = A.dot(mu_x)
            Sigma_x = A.dot(Sigma_x).dot(A.T) +C.dot(C.T)
            
    def stationary_distributions(self, max_iter=200, tol=1e-5):
        """
        compute the moments of the stationary distributions of x_t and
        y_t if possible. computation is by iteration, starting from the
        initial conditions self.mu_0 and self.Sigma_0.
        
        parameters
        ------
        max_iter: scalar(int), optional(default=200)
            the maximum number of iterations allowed
        tol: scalar(float), optional(default=1e-5)
            the tolerance level that one wishes to achieve
            
        returns
        ------
        mu_x_star: array_like(float)
            an n x 1 array representing the stationary mean of x_t
        mu_y_star: array_like(float)
            a k x 1 array representing the stationary mean of y_t
        Sigma_x_star: array_like(float)
            an n x n array representing the stationary var-cov matrix
            of x_t
        Sigma_y_star: array_like(float)
            a k x k array representing the stationary var-cov matrix
            of y_t
            
        """
        #==initialize iteration==#
        m = self.moment_sequence()
        mu_x, mu_y, Sigma_x, Sigma_y =next(m)
        i = 0
        error = tol + 1
        
        #==loop until convergence or failure==#
        while error > tol:
            
            if i > max_iter:
                fail_message = 'convergence failed after {} iterations'
                raise ValueError(fail_message.format(max_iter))
                
            else: 
                i += 1
                mu_x1, mu_y1, Sigma_x1, Sigma_y1 = next(m)
                error_mu = np.max(np.abs(mu_x1 - mu_x))
                error_Sigma = np.max(np.abs(Sigma_x1 - Sigma_x))
                error = max(error_mu, error_Sigma)
                mu_x, Sigma_x = mu_x1, Sigma_x1
                
        #== prepare return values==#
        mu_x_star, Sigma_x_star = mu_x, Sigma_x
        mu_y_star, Sigma_y_star = mu_y1, Sigma_y1
        
        return mu_x_star, mu_y_star, Sigma_x_star, Sigma_y_star
    
    def geometric_sums(self, beta, x_t):
        """
        forecast the geometric sums
        
            S_x := E [sum_{j=0}^{\infty} beta^j x_{t+j} | x_t ]
            
            S_y := E [sum_{j=0}^{\infty} beta^j y_{t+j} | x_t ]
            
        parameters
        ------
        beta: scalar(float)
            discount factor, in [0, 1)
        
        x_t: array_like(float)
            the term x_t for conditioning
            
        returns
        ------
        S_x: array_like(float)
            Geometric sum as defined above
            
        S_y: array_like(float)
            geometric sum as defined above
            
        """
        I = np.identity(self.n)
        S_x = solve(I - beta * self.A, x_t)
        S_y = self.G.dot(S_x)
        
        return S_x, S_y
    
    def impulse_response(self, j=5):
        """
        pulls off the impulse response coefficients to a shock
        in w_{t} for x and y
        
        important to note: we are uninterested in the shocks to 
        v for this method
        
        * x coefficients are C, A C, A^2 C ...
        * y coefficients are GC, GAC, GA^2 C...
        
        parameters
        ------
        j: scalar(int)
            number of coefficients that we want
        
        returns
        ------
        xcoef: list(array_like(float, 2))
            the coefficients for x
        ycoef: list(array_list(float, 2))
            the coefficients for y
            
        """
        #pull out matrices
        A, C, G, H =  self.A, self.C, self.G, self.H
        Apower = np.copy(A)
        
        #create room for coefficients
        xcoef = [C]
        ycoef = [np.dot(G, C)]
        
        for i in range(j):
            xcoef.append(np.dot(Apower, C))
            ycoef.append(np.dot(G, np.dot(Apower, C)))
            Apower = np.dot(Apower, A)
        
        return xcoef, ycoef